In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from load import load_pseudo, load_condons

pd.options.display.precision = 3
pd.options.display.max_colwidth = 10

%matplotlib inline

In [2]:
%time records = load_pseudo()
records

CPU times: user 5.22 s, sys: 106 ms, total: 5.33 s
Wall time: 5.35 s


,id,sequence,missing,missing_%,sequence_i,missing_i,missing_%_i,carb,toby
0,TA151,ATGAGT...,31842,6.588,ATGAGT...,28410,5.878,True,False
1,IC1,ATGAGT...,46071,9.532,ATGAGT...,34714,7.182,False,False
2,A237,ATGAGT...,44514,9.210,ATGAGT...,35933,7.434,True,False
3,5920,ATGAGT...,49497,10.241,ATGAGT...,36873,7.629,NaN,NaN
4,LiA96,ATGAGT...,44067,9.117,ATGAGT...,34454,7.128,False,False
...,...,...,...,...,...,...,...,...,...
117,JD318,------...,77766,16.090,ATGAGT...,39108,8.091,False,False
118,Jp238,------...,43062,8.909,ATGAGT...,32466,6.717,False,False
119,Jp1303,------...,44151,9.135,ATGAGT...,32792,6.785,False,False
120,JD304,------...,75465,15.613,ATGAGT...,38729,8.013,False,False


In [3]:
# 22 seconds
# %time o_c = load_condons('../data/pseudo/concatenated.fasta')
# %time i_c = load_condons('../data/pseudo/concatenated_naive_impute.fasta')

# Feature selection

In [4]:
# 1.5 minutes
# d = {}
# for label, content in o_c.iteritems():
#     d.update(content.value_counts().to_dict())
# d_sorted = dict(sorted(d.items(), key=lambda x: x[1], reverse=True))
# mapping = {key: i for i, key in enumerate(d_sorted.keys())}

# import json
# with open('../data/pseudo/preprocess/others/condon_mapping.json', 'w') as output:
#     json.dump(mapping, output, indent='\t')

In [5]:
import json
with open('../data/pseudo/preprocess/others/condon_mapping.json', 'r') as input_:
    mapping = json.load(input_)

In [6]:
# 22 seconds
# %time o_c_ = o_c.applymap(lambda x: mapping[x])
# %time i_c_ = i_c.applymap(lambda x: mapping[x])
# np.save('../data/pseudo/preprocess/o_c_-.npy', o_c_)
# np.save('../data/pseudo/preprocess/i_c_-.npy', i_c_)

In [3]:
o_c_ = np.load('../data/pseudo/preprocess/o_c_-.npy')
i_c_ = np.load('../data/pseudo/preprocess/i_c_-.npy')

In [8]:
mask = (records['toby'].notna() & records['carb'].notna())

## Remove based on SNP counts
similar to variance threshold but seems better

In [9]:
# 2 minutes
# %time variant_counts_o = o_c.apply(pd.Series.value_counts, axis=0)
# %time variant_counts_i = i_c.apply(pd.Series.value_counts, axis=0)
# np.save('../data/pseudo/preprocess/others/variant_counts_o.npy', variant_counts_o)
# np.save('../data/pseudo/preprocess/others/variant_counts_i.npy', variant_counts_i)

In [10]:
variant_counts_o = pd.DataFrame(np.load('../data/pseudo/preprocess/others/variant_counts_o.npy'))
variant_counts_i = pd.DataFrame(np.load('../data/pseudo/preprocess/others/variant_counts_i.npy'))

In [11]:
variant_max_counts_o = variant_counts_o.max()
(pd.Series(variant_max_counts_o<121)).value_counts()

True     85753
False    75358
dtype: int64

In [12]:
variant_max_counts_i = variant_counts_i.max()
(variant_max_counts_i<121).value_counts()

False    104920
True      56191
dtype: int64

In [13]:
o_c_v = o_c_[mask][:, variant_max_counts_o<121]
i_c_v = i_c_[mask][:, variant_max_counts_i<121]

In [14]:
np.save('../data/pseudo/preprocess/o_c_v.npy', o_c_v)
np.save('../data/pseudo/preprocess/i_c_v.npy', i_c_v)

## $\chi^2$ on the previous result
because some features are all 0's, so gives `divide by 0` warning

no warning if we remove those features (on the previous step)

In [15]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

In [16]:
kbest_o = SelectKBest(chi2, k=85753//2).fit_transform(o_c_v, records['toby'][mask].astype('i4'))
kbest_o.shape

(119, 42876)

In [17]:
kbest_i = SelectKBest(chi2, k=56191//2).fit_transform(i_c_v, records['toby'][mask].astype('i4'))
kbest_i.shape

(119, 28095)

In [18]:
np.save('../data/pseudo/preprocess/o_c_x.npy', kbest_o)
np.save('../data/pseudo/preprocess/i_c_x.npy', kbest_i)

# Feature extraction

## String kernel

In [4]:
from strkernel.mismatch_kernel import MismatchKernel

In [ ]:
%time o_c__s = MismatchKernel(l=125, k=4, m=1).get_kernel(o_c_)

In [ ]:
%time i_c__s = MismatchKernel(l=125, k=4, m=1).get_kernel(i_c_)